In [0]:
%sql
USE japan_gold;

WITH
-- ==========================================
-- Base POS scope for all manufacturers (no brand filter)
-- ==========================================
pos_scope AS (
  SELECT
    pos.pos_sales_amt,
    prod.jp_mfgr_name,
    TRIM(prod.jp_local_pos_share_flag) AS jp_local_pos_share_flag,
    prod.jp_sub_brand_name,
    cust.jp_cust_lvl_1_name,
    cal.fy_month_name
  FROM pos_monthly_fct_vw pos
  JOIN site_dim_vw          site ON site.site_key = pos.site_key
  JOIN calender_dim_vw      cal  ON cal.day_date = pos.day_date
  JOIN product_dim_vw       prod ON prod.prod_key = pos.prod_key
  JOIN customer_japan_dim_vw cust ON cust.cust_key = site.own_party_key
  WHERE cust.jp_cust_lvl_1_name = 'MATSUKIYOCOCOKARA&COMPANY -JP'
),

-- ==========================================
-- Base POS scope filtered to P&G only
-- Used for the "Value Share (P&G)" that must show 100%
-- ==========================================
pos_scope_pg AS (
  SELECT
    pos.pos_sales_amt,
    TRIM(prod.jp_local_pos_share_flag) AS jp_local_pos_share_flag,
    prod.jp_sub_brand_name,
    cal.fy_month_name
  FROM pos_monthly_fct_vw pos
  JOIN site_dim_vw          site ON site.site_key = pos.site_key
  JOIN calender_dim_vw      cal  ON cal.day_date = pos.day_date
  JOIN product_dim_vw       prod ON prod.prod_key = pos.prod_key
  JOIN customer_japan_dim_vw cust ON cust.cust_key = site.own_party_key
  WHERE cust.jp_cust_lvl_1_name = 'MATSUKIYOCOCOKARA&COMPANY -JP'
    AND prod.jp_mfgr_name = 'P&G'
),

-- MCC POS YA (2024M04–2024M09) - all manufacturers
pos_ya AS (
  SELECT
    'MCC' AS mcc,
    SUM(pos_sales_amt) AS pos_ya
  FROM pos_scope
  WHERE fy_month_name BETWEEN '2024M04' AND '2024M09'
),

-- MCC Value Share (three players P&G, KAO, LION) - all manufacturers, correct share
value_share AS (
  SELECT
    'MCC' AS mcc,
    100.0 *
      SUM(CASE WHEN jp_local_pos_share_flag = 'in'
                AND jp_mfgr_name IN ('P&G','KAO','LION')
               THEN pos_sales_amt ELSE 0 END)
      / NULLIF(
          SUM(CASE WHEN jp_local_pos_share_flag = 'in'
                   THEN pos_sales_amt ELSE 0 END),
          0
        ) AS value_share
  FROM pos_scope
  WHERE fy_month_name BETWEEN '2024M04' AND '2024M09'
),

-- MCC POS YA (P&G) — previous year window 2023M04–2023M09
pos_ya_pg AS (
  SELECT
    'MCC' AS mcc,
    SUM(pos_sales_amt) AS pos_ya_pg
  FROM pos_scope
  WHERE fy_month_name BETWEEN '2023M04' AND '2023M09'
    AND jp_mfgr_name = 'P&G'
),

-- ==========================================
-- Value Share (P&G) computed on dataset filtered to P&G only
-- Returns 100% by design (numerator == denominator)
-- ==========================================
value_share_pg_100 AS (
  SELECT
    'MCC' AS mcc,
    100.0 *
      SUM(CASE WHEN jp_local_pos_share_flag = 'in' THEN pos_sales_amt ELSE 0 END)
      / NULLIF(
          SUM(CASE WHEN jp_local_pos_share_flag = 'in' THEN pos_sales_amt ELSE 0 END),
          0
        ) AS value_share_pg
  FROM pos_scope_pg
  WHERE fy_month_name BETWEEN '2024M04' AND '2024M09'
),

-- Store SoDF (Apr–Sep 2024), Display, RT parent group (original scope; numerator P&G)
sodf AS (
  SELECT
    'MCC' AS mcc,
    100.0 *
      SUM(
        CASE
          WHEN TRIM(p.jp_mfgr_name) = 'P&G'
           AND TRIM(p.jp_local_pos_share_flag) = 'in'
           AND TRIM(p.jp_local_trax_flag)      = 'in'
          THEN k.measure_qty
          ELSE 0
        END
      )
      / NULLIF(
          SUM(
            CASE
              WHEN TRIM(p.jp_mfgr_name) NOT IN ('Others','General','')
               AND TRIM(p.jp_local_trax_flag) = 'in'
              THEN k.measure_qty
              ELSE 0
            END
          ),
          0
        ) AS sodf
  FROM kpi_qty_fct_vw k
  JOIN product_dim_vw        p    ON k.prod_key = p.prod_key
  JOIN placement_dim_vw      pl   ON pl.placement_key = k.placement_key
  JOIN site_dim_vw           site ON k.site_key = site.site_key
  JOIN customer_japan_dim_vw cust ON site.own_party_key = cust.cust_key
  WHERE k.measure_name = 'PGJAPAN_SKU_FACINGS_BY_SCENE --- result'
    AND CAST(k.day_date AS DATE) BETWEEN DATE('2024-04-01') AND DATE('2024-09-30')
    AND pl.placement_type_name = 'Display'
    AND cust.jp_cust_parent_group_name = 'RT'
    AND cust.jp_cust_lvl_1_name = 'MATSUKIYOCOCOKARA&COMPANY -JP'
),

-- Initiative POS Distribution (Spring + Summer 2024) for MCC
pos_distribution AS (
  SELECT
    'MCC' AS mcc,
    SUM(pos.pos_distribution) AS pos_distribution
  FROM initiative_pos_distribution_actual_vw pos
  JOIN customer_japan_dim_vw cust ON cust.jp_cust_id = pos.fn_code
  JOIN initiative_master_vw im     ON im.initiative_key = pos.sl_initiative_key
  WHERE time_period IN ('2024 Spring','2024 Summer')
    AND cust.jp_cust_name = 'FN-MATSUKIYO_COCOKARA (B008)'
),

-- Value Share (Ariel) — all manufacturers scoped to sub-brand
value_share_ariel AS (
  SELECT
    'MCC' AS mcc,
    100.0 *
      SUM(CASE WHEN TRIM(prod.jp_local_pos_share_flag) = 'in'
                AND prod.jp_mfgr_name IN ('P&G','KAO','LION')
               THEN pos.pos_sales_amt ELSE 0 END)
      / NULLIF(
          SUM(CASE WHEN TRIM(prod.jp_local_pos_share_flag) = 'in'
                   THEN pos.pos_sales_amt ELSE 0 END),
          0
        ) AS value_share_ariel
  FROM pos_monthly_fct_vw pos
  JOIN product_dim_vw       prod ON prod.prod_key = pos.prod_key
  JOIN calender_dim_vw      cal  ON cal.day_date = pos.day_date
  JOIN site_dim_vw          site ON site.site_key = pos.site_key
  JOIN customer_japan_dim_vw cust ON cust.cust_key = site.own_party_key
  WHERE cust.jp_cust_lvl_1_name = 'MATSUKIYOCOCOKARA&COMPANY -JP'
    AND cal.fy_month_name BETWEEN '2024M04' AND '2024M09'
    AND prod.jp_sub_brand_name = 'Ariel'
),

-- Value Share (Ariel Gel Ball) — all manufacturers scoped to sub-brand
value_share_ariel_gb AS (
  SELECT
    'MCC' AS mcc,
    100.0 *
      SUM(CASE WHEN TRIM(prod.jp_local_pos_share_flag) = 'in'
                AND prod.jp_mfgr_name IN ('P&G','KAO','LION')
               THEN pos.pos_sales_amt ELSE 0 END)
      / NULLIF(
          SUM(CASE WHEN TRIM(prod.jp_local_pos_share_flag) = 'in'
                   THEN pos.pos_sales_amt ELSE 0 END),
          0
        ) AS value_share_ariel_gel_ball
  FROM pos_monthly_fct_vw pos
  JOIN product_dim_vw       prod ON prod.prod_key = pos.prod_key
  JOIN calender_dim_vw      cal  ON cal.day_date = pos.day_date
  JOIN site_dim_vw          site ON site.site_key = pos.site_key
  JOIN customer_japan_dim_vw cust ON cust.cust_key = site.own_party_key
  WHERE cust.jp_cust_lvl_1_name = 'MATSUKIYOCOCOKARA&COMPANY -JP'
    AND cal.fy_month_name BETWEEN '2024M04' AND '2024M09'
    AND prod.jp_sub_brand_name = 'Ariel Gel Ball'
)

-- ==========================================
-- Final single-row, multi-column output
-- ==========================================
SELECT
  'MCC'                                           AS MCC,
  pos_ya.pos_ya                                   AS `POS YA`,
  value_share.value_share                         AS `Value Share`,
  pos_ya_pg.pos_ya_pg                             AS `POS YA (P&G)`,
  value_share_pg_100.value_share_pg               AS `Value Share (P&G)`,
  sodf.sodf                                       AS `Store SoDF`,
  pos_distribution.pos_distribution               AS `POS Distribution`,
  value_share_ariel.value_share_ariel             AS `Value Share (Ariel)`,
  value_share_ariel_gb.value_share_ariel_gel_ball AS `Value Share (Ariel Gel ball)`
FROM pos_ya
CROSS JOIN value_share
CROSS JOIN pos_ya_pg
CROSS JOIN value_share_pg_100
CROSS JOIN sodf
CROSS JOIN pos_distribution
CROSS JOIN value_share_ariel
CROSS JOIN value_share_ariel_gb;
